In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
from utils.data_handling import *
from utils.sensor_optimisation import *
from parameters import *
import GPy
import scipy.stats
from sklearn.covariance import *
from utils.data_assimilation import *
import time



import matplotlib.pyplot as plt
np.random.seed(101)



### Importing the Tracer Data to Select the Optimisation set S

In [6]:
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0, 'i_end': 988, 'crop': None, 'field_name': 'Tracer'}

In [7]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  6.23it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


### Working subset of the data : set S

In [8]:
S_human = human_level_subset(buildingshape, loc_df, 2,1 )
print("The remaining number of points is : ", str(len(S_human)))

The remaining number of points is :  37847


In [9]:
S_working = working_subset(data_df, loc_df, nbins = (25,25,25), threshold_sum = 10**-2 )
print("The remaining number of points is : ", str(len(S_working)))

The remaining number of points is :  57725


In [10]:
S = np.intersect1d(S_human,S_working)
print("The remaining number of points is : ", str(len(S)))

The remaining number of points is :  23643


In [11]:
#A_list = set_to_onehot(S,data_df.shape[0])
#save_vtu(ref_vtu[0],'FinalSelection',A_list)

### Import the Field of interest for computing the optimisation : TracerBackground / Pressure

In [12]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0, 'i_end': 988, 'crop': None, 'field_name': 'Tracer'}

In [13]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

  0%|          | 0/1 [00:00<?, ?it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


100%|██████████| 1/1 [00:00<00:00,  6.61it/s]


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


In [14]:
X = loc_df.values[:,:] #[I,:dim]
Z = data_df.values[:,:] #[:,t].reshape(-1,1) # [I,t].reshape(-1,1)
Z.shape

(100040, 989)

## Data Assimilation 

In [16]:
indLoc = np.loadtxt('Optimal Points/localpoints_optimal_eps10-6.txt').astype(int)
MSExb, MSExDA = data_assimilation(indLoc,Z)
MSExb - MSExDA

[ 3078 10357 19782 43278 52731 54786 55640 56257 56588 73959]
L2 norm of the background error 0.9923911506964529
L2 norm of the error in DA solution 5.876031868112651e-17


0.9923911506964528

In [20]:
s = time.time()
#A_ball = ball_set(indLoc, loc_df,radius=5)
indLoc_rand = randomise_set(A_ball)   
data_assimilation(indLoc_rand,Z) 
t = time.time() - s
t

[58838, 9837, 20574, 85597, 2572, 26206, 12544, 54528, 22293, 87636]
L2 norm of the background error 0.6049754148695947
L2 norm of the error in DA solution 0.015035791575613666


15.4012291431427

In [21]:
n_rand = 500
DA_results_rand = dict()
#A_ball = ball_set(indLoc, loc_df,radius=5)
for n in tqdm.tqdm(range(n_rand)):
    DA_results_rand[n] = dict()
    s = time.time()
    DA_results_rand[n]['indLoc'] = randomise_set(A_ball)   
    DA_results_rand[n]['MSExb'], DA_results_rand[n]['MSExDA'] = data_assimilation(DA_results_rand[n]['indLoc'],Z)
    DA_results_rand[n]['time'] = time.time() - s

[62866, 9837, 22736, 1664, 16255, 18411, 35221, 21857, 10439, 78628]
L2 norm of the background error 0.6320603272206871
L2 norm of the error in DA solution 0.09346576965729196
[75123, 1502, 28032, 69012, 16019, 31764, 3332, 33009, 10356, 78628]
L2 norm of the background error 1.2247395803709027
L2 norm of the error in DA solution 0.02916366837655588
[56389, 9837, 27617, 39303, 3361, 62301, 55485, 89353, 12322, 69063]
L2 norm of the background error 0.5865973849530034
L2 norm of the error in DA solution 0.03223237487396566
[53134, 6525, 20336, 8222, 15024, 27429, 14722, 14481, 19255, 50700]
L2 norm of the background error 0.6881997697845642
L2 norm of the error in DA solution 0.08370984425512817
[32853, 2293, 43775, 24176, 2569, 12527, 55485, 9621, 8412, 54412]
L2 norm of the background error 0.9753746153589178
L2 norm of the error in DA solution 0.0794698755496644
[62866, 6525, 78124, 36269, 1307, 27429, 2593, 68418, 22293, 28937]
L2 norm of the background error 0.6241019457182442
L2 n

[22259, 34112, 73960, 36269, 10495, 12527, 956, 9621, 8412, 50307]
L2 norm of the background error 0.9214714298340898
L2 norm of the error in DA solution 0.31504322153369774
[56389, 1502, 25519, 1664, 1268, 18411, 2593, 48384, 12048, 53188]
L2 norm of the background error 0.44752898023847415
L2 norm of the error in DA solution 0.0623885182546183
[75123, 12849, 17616, 6637, 15024, 24623, 5180, 48792, 1501, 4898]
L2 norm of the background error 0.8117358008715319
L2 norm of the error in DA solution 0.2162874721851334
[60397, 34112, 59215, 8222, 16019, 26206, 35221, 68841, 58368, 28937]
L2 norm of the background error 0.7722095323731347
L2 norm of the error in DA solution 0.00842144275931152
[21540, 2856, 8850, 38494, 52622, 17183, 7259, 7148, 22371, 33545]
L2 norm of the background error 0.5807843877292785
L2 norm of the error in DA solution 0.01511307565849962
[60538, 17017, 24738, 6637, 16019, 8741, 12544, 66617, 29094, 32908]
L2 norm of the background error 0.49009961619694664
L2 norm

[37099, 8473, 25017, 39360, 13654, 30834, 6189, 16869, 12119, 81]
L2 norm of the background error 1.064048528397968
L2 norm of the error in DA solution 0.013447234953498274
[37099, 1502, 37760, 18688, 3112, 27429, 14722, 68841, 6525, 18722]
L2 norm of the background error 1.021557233186097
L2 norm of the error in DA solution 0.10336080578215034
[53134, 12119, 28032, 48511, 11301, 12527, 40407, 76468, 8477, 87244]
L2 norm of the background error 0.9261957612948722
L2 norm of the error in DA solution 0.16906403277393764
[20458, 60179, 22736, 25842, 4009, 29987, 6305, 16869, 58368, 28104]
L2 norm of the background error 0.9434393962473065
L2 norm of the error in DA solution 0.19049797200642044
[14481, 60179, 20574, 53139, 1306, 4693, 991, 74845, 22371, 23438]
L2 norm of the background error 0.8292007672933128
L2 norm of the error in DA solution 0.07767986407767818
[62866, 58368, 78414, 49119, 10495, 14677, 956, 15695, 12322, 28937]
L2 norm of the background error 0.8479880960575866
L2 nor

[43758, 1990, 78150, 39303, 11301, 17184, 47531, 57409, 35086, 87244]
L2 norm of the background error 0.7374226251830271
L2 norm of the error in DA solution 0.04153430633809156
[26467, 22293, 17616, 53139, 1306, 7097, 3332, 53480, 14748, 74629]
L2 norm of the background error 1.2977443401735633
L2 norm of the error in DA solution 0.05595519541058103
[7183, 8473, 8850, 8222, 16256, 12527, 35221, 53480, 8412, 60246]
L2 norm of the background error 1.0154928184860628
L2 norm of the error in DA solution 0.24901704253150922
[14481, 1502, 27617, 76609, 13654, 17184, 40011, 28558, 35086, 53198]
L2 norm of the background error 0.6953219028762533
L2 norm of the error in DA solution 0.07087348393451542
[14047, 1498, 17088, 16356, 6304, 8741, 40011, 65623, 1502, 43793]
L2 norm of the background error 1.1054536375844415
L2 norm of the error in DA solution 0.011601622477525253
[19523, 55952, 42413, 39303, 1307, 18411, 7547, 85866, 2293, 64537]
L2 norm of the background error 0.7394123996139356
L2 n

[76517, 6525, 31465, 8223, 10495, 44088, 956, 65851, 14748, 69878]
L2 norm of the background error 0.8064240650672113
L2 norm of the error in DA solution 0.10940787197254884
[92076, 24950, 25017, 34785, 11724, 85857, 14722, 53298, 2292, 87748]
L2 norm of the background error 0.8901972458993873
L2 norm of the error in DA solution 0.11209194587369072
[78627, 52730, 88233, 18688, 6303, 11198, 2593, 21750, 1498, 75169]
L2 norm of the background error 0.768611592270095
L2 norm of the error in DA solution 0.039077125848431454
[14481, 22293, 1930, 6637, 4009, 17183, 3332, 80932, 58368, 81]
L2 norm of the background error 0.6937954345414059
L2 norm of the error in DA solution 0.16444241336862628
[22259, 55952, 37760, 76609, 16256, 4693, 2593, 7148, 9837, 1447]
L2 norm of the background error 0.8390594497805428
L2 norm of the error in DA solution 0.08660111546944302
[709, 58368, 78414, 78177, 16255, 85857, 40011, 26045, 9837, 18722]
L2 norm of the background error 0.48167892221338376
L2 norm of

...

[52986, 17017, 62048, 8223, 4009, 27429, 3332, 65623, 22293, 69878]
L2 norm of the background error 0.9007408647960147
L2 norm of the error in DA solution 0.08030532087816318
[78776, 2293, 62048, 85597, 10495, 26206, 7547, 55231, 12048, 26499]
L2 norm of the background error 0.8046309054961335
L2 norm of the error in DA solution 0.02998605808678945
[43853, 19616, 73958, 11087, 16019, 29987, 7547, 65851, 14748, 30046]
L2 norm of the background error 0.8623579811347879
L2 norm of the error in DA solution 0.2838256627074265
[44150, 6525, 27617, 8584, 11724, 24623, 2593, 25823, 1501, 62305]
L2 norm of the background error 0.5821409501124317
L2 norm of the error in DA solution 0.0708628692711917
[55803, 12119, 73958, 48511, 13403, 14677, 40557, 42813, 1502, 52390]
L2 norm of the background error 0.805677077530251
L2 norm of the error in DA solution 0.13309489493157484
[49103, 22371, 27859, 90395, 3581, 30834, 7547, 85866, 9837, 4898]
L2 norm of the background error 0.7906042654191344
L2 nor

[89500, 19616, 49600, 38729, 3112, 27429, 7547, 78208, 1502, 58456]
L2 norm of the background error 0.9318330303742695
L2 norm of the error in DA solution 0.06284035281384244
[43758, 55952, 90624, 8584, 1992, 49667, 7259, 75098, 22371, 62305]
L2 norm of the background error 0.8868223257597525
L2 norm of the error in DA solution 0.08059747864509459
[14481, 60179, 78414, 21267, 52622, 12527, 55485, 24134, 6525, 53567]
L2 norm of the background error 0.582626489299342
L2 norm of the error in DA solution 0.17829512253878962
[22259, 2293, 73957, 49120, 2569, 11198, 40407, 60059, 35086, 55629]
L2 norm of the background error 0.981723053021784
L2 norm of the error in DA solution 0.03672211077101016
[16871, 1498, 49437, 39360, 52622, 8741, 7259, 66617, 10439, 18722]
L2 norm of the background error 0.7474480527817872
L2 norm of the error in DA solution 0.04860793655686476
[76517, 34112, 20574, 8222, 11724, 18411, 12544, 52342, 58368, 87636]
L2 norm of the background error 0.7182892904656031
L2 

[58839, 2856, 29626, 8584, 851, 18411, 43412, 59426, 12322, 17961]
L2 norm of the background error 0.8179319987248105
L2 norm of the error in DA solution 0.2924216480026682
[11537, 9837, 90376, 37962, 1992, 24623, 40557, 24134, 1498, 30046]
L2 norm of the background error 0.775071657489684
L2 norm of the error in DA solution 0.023322282052301458
[48185, 52730, 29647, 39303, 851, 21124, 6189, 79403, 22371, 43793]
L2 norm of the background error 0.612786029503019
L2 norm of the error in DA solution 0.03711523555592398
[60397, 58368, 43360, 11087, 851, 11198, 12544, 60059, 22293, 68566]
L2 norm of the background error 0.9833793439812252
L2 norm of the error in DA solution 0.027317528109722877
[24347, 22293, 88233, 36269, 13403, 23096, 6189, 30349, 22371, 52390]
L2 norm of the background error 0.7131567760379366
L2 norm of the error in DA solution 0.15386816296266603
[14514, 21604, 20336, 36269, 52622, 27429, 3332, 26045, 10439, 69878]
L2 norm of the background error 0.7537006016856055
L2 

[11856, 22293, 22733, 85597, 3581, 23096, 14722, 52357, 6872, 23438]
L2 norm of the background error 0.5908321985901871
L2 norm of the error in DA solution 0.007224418163200497
[69636, 34112, 59215, 30407, 16019, 36195, 991, 47075, 1498, 81]
L2 norm of the background error 0.6277899625172139
L2 norm of the error in DA solution 0.0196236867249531
[14481, 21604, 25519, 78177, 3112, 21124, 3332, 89353, 8412, 75169]
L2 norm of the background error 0.6851893752097951
L2 norm of the error in DA solution 0.15110537868659749
[42226, 2856, 44029, 25842, 2572, 23096, 5180, 55231, 58367, 23327]
L2 norm of the background error 1.0361629140951965
L2 norm of the error in DA solution 0.2206772074363255
[43758, 52730, 29645, 23338, 15024, 44088, 3332, 7148, 10439, 52390]
L2 norm of the background error 0.9399866694796922
L2 norm of the error in DA solution 0.10824572904347243
[53134, 12119, 78414, 21399, 13654, 27429, 3332, 58970, 35086, 44539]
L2 norm of the background error 0.84650729775771
L2 norm 

[89500, 19616, 1930, 44643, 3361, 4693, 35221, 30511, 1498, 50307]
L2 norm of the background error 1.0270814108749136
L2 norm of the error in DA solution 0.08475951571197854
[78627, 24950, 27859, 9740, 15024, 7097, 40557, 30349, 22371, 58456]
L2 norm of the background error 0.807233503043899
L2 norm of the error in DA solution 0.02990803020506332
[75050, 6525, 49600, 36269, 16255, 31764, 35221, 43106, 12119, 87636]
L2 norm of the background error 0.7954007161923328
L2 norm of the error in DA solution 0.24929572019809668
[48385, 10786, 20336, 31465, 11724, 24623, 3332, 47075, 8412, 587]
L2 norm of the background error 0.7687280925647202
L2 norm of the error in DA solution 0.1730629117262358
[29791, 10786, 90384, 8222, 16019, 27429, 43412, 89353, 6872, 4898]
L2 norm of the background error 0.9863651176276883
L2 norm of the error in DA solution 0.37944106225468655
[48115, 12849, 27617, 39303, 6304, 31764, 5180, 78208, 22293, 64537]
L2 norm of the background error 0.5944873898520902
L2 nor

In [39]:
MSExDA_mean = []
MSExDA_std = 0
MSExb_mean = []
MSExb_std = 0
MSE_xDA_b = []
MSE_xDA_b_mean = 0
MSE_xDA_b_std = 0

for i in DA_results_rand:
    MSExDA_mean.append(DA_results_rand[i]['MSExDA'])
    MSExb_mean.append(DA_results_rand[i]['MSExb'])
    MSE_xDA_b.append(DA_results_rand[i]['MSExb'] - DA_results_rand[i]['MSExDA'])

MSExDA_std = np.std(MSExDA_mean)
MSExDA_mean = np.mean(MSExDA_mean)

MSExb_std = np.std(MSExb_mean)
MSExb_mean = np.mean(MSExb_mean)

MSE_xDA_b_std = np.std(MSE_xDA_b)
MSE_xDA_b_mean = np.mean(MSE_xDA_b)


In [40]:
MSExDA_mean, MSExDA_std

0.11070491523762266

In [41]:
MSExb_mean, MSExb_std

0.8298164985082452

In [42]:
MSE_xDA_b_mean, MSE_xDA_b_std

0.7191115832706225

In [ ]:
MSExDA_mean / MSExDA